In [ ]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans

In [82]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [61]:
executor_mem = "150g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'

In [53]:
# this is where the magic happens
results = [] 
num_filters = 1024
gamma = 0.1
res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           filter_gen_gen=make_gaussian_filter_gen,
                           cores_per_executor=cores_per_executor,
                           num_filters=num_filters,
                           sample=1.0,
                           gamma=gamma,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput="/user/vaishaal/featurizedWindows/sequence-only/full/{0}-filters/{1}-gamma/replicate-0".format(num_filters, gamma),
                           num_executors=num_executors)
results.append(res)

(1024, 32)


In [62]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])

In [83]:
results_df = cross_validate("/user/vaishaal/featurizedWindows/sequence-only/full/{0}-filters/{1}-gamma/replicate-0".format(num_filters, gamma), 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001, 0.0001],
                            regs=[1e-2])
    

FOLD 0
HOLDING OUT CHROMOSOMES ['chr12']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=0.01

part-00035
part-00853
part-00504
part-00168
part-00481
part-00220
part-00502
part-00743
part-00375
part-00534
part-00075
part-00658
part-00754
part-00239
part-00829
part-00090
part-00354
part-00238
part-00611
part-00603
part-00126
part-00749
part-00202
part-00616
part-00597
part-00819
part-00306
part-00762
part-00242
part-00655
part-00524
part-00792
part-00143
part-00387
part-00386
part-00799
part-00235
part-00314
part-00241
part-00463
part-00207
part-00151
part-00689
part-00563
part-00358
part-00875
part-00545
part-00118
part-00846
part-00056
part-00266
part-00458
part-00084
part-00197
part-00350
part-00684
part-00184
part-00734
part-00552
part-00737
part-00520
part-00681
part-00011
part-00530
part-00274
part-00156
part-00866
part-00526
part-00287
part-00539
part-00680
part-00189
part-00060
part-00334
part-00297
part-00547
part-00774
part-00015
part-00083
part-00779
part-00507
part-00256
p

In [68]:
old = results_df

In [84]:
results_df

,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.0010,0.01,[3],[chr12],0.954726,0.969473,0.736193,0.856400,0.982679,0.998876,0.035594,0.903343,0,0,0,0
1,0.0001,0.01,[3],[chr12],0.993585,0.994914,0.998978,0.999171,0.999695,1.000000,0.017933,0.884693,0,0,0,0
